In [28]:
import pandas as pd
import requests
import bs4

from random import randint
from time import sleep

In [154]:
headers = {
    'Host': 'market.yandex.ru',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.6,en;q=0.4'
}

In [ ]:
hids = [7023082, 7835734, 8473449, 10495456, 11028554, 1716316475, 1720465387, 11028534, 12466715, 13485518, 13934959, 
        12259333, 12340881, 11028576, 12847672, 13579073, 12089499, 10724875, 12750156, 12923930, 13934961, 11001944, 
        9323459, 5115280, 13188751, 12772728, 10725078, 6258760, 8440056, 13485515, 13340782, 12748971, 10546687, 
        10404627, 12672696, 9383775, 10967066, 13188749, 7752791, 10725082, 14124403, 12206544, 13471372, 12840987]

In [231]:
for hid in hids:
    for grade in range(1, 6): 
        page = 1
        visited = False
        text = []
        ids = set()

        print('Grade = {}'.format(grade))

        while not visited:
            url = 'https://market.yandex.ru/product/{}/reviews?grade_value={}&page={}'.format(hid, grade, page)
            request = requests.get(url, headers=headers)
            soup = bs4.BeautifulSoup(request.text, 'lxml')

            items = soup.find_all('div', attrs={'class': 'n-product-review-item'})

            print('Lenght of ids {}'.format(len(items)))

            if len(items) == 0:
                print('Maybe I\'m blocked')
                visited = True
                break

            for t in items:
                if 'data-review-id' in t.attrs:
                    id = t.attrs['data-review-id']
                    if id in ids:
                        visited = True
                        break

                    ids.add(id)

            if visited:
                break
                
            
            if len(items) < 10:
                visited = True
                

            for t in soup.find_all('meta', attrs={'itemprop': 'description'}):
                if 'content' in t.attrs:
                    text.append(t.attrs['content'])


            print('Page {}'.format(page))
            page+=1
            sleep(randint(10, 20))


        filename = 'data/reviews_{}_{}.csv'.format(hid, grade) 
        df = pd.DataFrame({'text':text, 'class': [grade] * len(text)})
        df.to_csv(filename)

        print('{} is finished. Lenght {}'.format(filename, df.shape[0]))

Grade = 2
Lenght of ids 10
Page 1
Lenght of ids 7
Page 2
data/reviews_12206544_2.csv is finished. Lenght 17
Grade = 2
Lenght of ids 10
Page 1
Lenght of ids 10
Page 2
Lenght of ids 1
Page 3
data/reviews_13471372_2.csv is finished. Lenght 21
Grade = 2
Lenght of ids 10
Page 1
Lenght of ids 8
Page 2
data/reviews_12840987_2.csv is finished. Lenght 18


In [232]:
df = pd.DataFrame({'text':[], 'class': []})

for id in hids:
    for g in range(2, 3):
        filename = 'data/reviews_{}_{}.csv'.format(id, g)
        temp = pd.read_csv(filename, usecols=['class', 'text'], encoding="windows-1254")
        df = df.append(temp[['class', 'text']])
        
df[['class', 'text']].to_csv('data/train_data_raw.csv', encoding="windows-1254")

print(df.shape)
print(df['class'].value_counts())

(5833, 2)
3.0    1216
4.0    1194
2.0    1171
1.0    1129
5.0    1123
Name: class, dtype: int64


In [265]:
df = df[df['class'] != 3]

df['class'] = df['class'].apply(pd.to_numeric)

df['new_class'] = [''] * df.shape[0] 

df['new_class'][df['class'] < 3] = 'neg'
df['new_class'][df['class'] > 3] = 'pos'

df['class'] = df['new_class']

print(df['class'].value_counts())

df[['class', 'text']].to_csv('data/train_data.csv')

pos    2317
neg    2300
Name: class, dtype: int64


C:\Users\Public\PythonProjects\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Public\PythonProjects\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
